In [150]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///alchemy_orm.db', echo=True)

In [151]:
from sqlalchemy.ext.declarative import declarative_base

base = declarative_base()

base를 상속받아 class를 만들어야 한다.

In [152]:
from sqlalchemy import Column, Integer, String, ForeignKey

In [153]:
class Artist(base):
    __tablename__ = 'artist'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    def __repr__(self):
        return "<T'artist(name='%s')>" % (self.name)

In [154]:
class Album(base):
    __tablename__ = 'album'
    
    id = Column(Integer, primary_key=True)
    title = Column(String)
    artist_id = Column(Integer, ForeignKey('artist.id'))

In [155]:
class Genre(base):
    __tablename__ = 'genre'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)

In [156]:
class Track(base):
    __tablename__ = 'track'
    
    id = Column(Integer, primary_key=True)
    title = Column(String)
    album_id = Column(Integer, ForeignKey('album.id'))
    genre_id = Column(Integer, ForeignKey('genre.id'))

일반적인 파이썬 코딩 작업과 매우 흡사하다.

In [157]:
base.metadata.create_all(engine)

2018-07-15 21:27:53,399 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-15 21:27:53,401 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 21:27:53,402 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-15 21:27:53,404 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 21:27:53,406 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("artist")
2018-07-15 21:27:53,407 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 21:27:53,410 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("album")
2018-07-15 21:27:53,412 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 21:27:53,413 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("genre")
2018-07-15 21:27:53,415 INFO sqlalchemy.engine.base.Engine ()
2018-07-15 21:27:53,418 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("track")
2018-07-15 21:27:53,419 INFO sqlalchemy.engine.base.Engine ()


class를 선언해서 넘겨주면 알아서 테이블을 만든다.

In [158]:
artist1 = Artist(name='Led Zepplin')
artist2 = Artist(name='AC/DC')

In [159]:
artist1

<T'artist(name='Led Zepplin')>

메모리 공간과 실제 공간이 분리되어 있다 => session을 사용하여 연결해줘야 한다.

In [160]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

base = core의 metadata, session = core의 conn

In [161]:
session.add_all([artist1, artist2])
session.commit()

2018-07-15 21:29:34,697 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-15 21:29:34,703 INFO sqlalchemy.engine.base.Engine INSERT INTO artist (name) VALUES (?)
2018-07-15 21:29:34,704 INFO sqlalchemy.engine.base.Engine ('Led Zepplin',)
2018-07-15 21:29:34,708 INFO sqlalchemy.engine.base.Engine INSERT INTO artist (name) VALUES (?)
2018-07-15 21:29:34,709 INFO sqlalchemy.engine.base.Engine ('AC/DC',)
2018-07-15 21:29:34,711 INFO sqlalchemy.engine.base.Engine COMMIT


session을 통해 바인딩된 엔진을 통해 실제 SQL을 실행하여 물리적으로 저장.

In [162]:
session.dirty

IdentitySet([])

In [163]:
artist2.name = 'xyz'

In [164]:
session.dirty

IdentitySet([<T'artist(name='xyz')>])

dirty
- 메모리 상에 올라온 인스턴스의 값이 바뀌거나 삭제된 것이 있는지 감시.
- 바뀐 것이 있으면 commit 해야 함을 알림

In [165]:
session.commit()

2018-07-15 21:30:11,034 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-15 21:30:11,039 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id 
FROM artist 
WHERE artist.id = ?
2018-07-15 21:30:11,041 INFO sqlalchemy.engine.base.Engine (14,)
2018-07-15 21:30:11,045 INFO sqlalchemy.engine.base.Engine UPDATE artist SET name=? WHERE artist.id = ?
2018-07-15 21:30:11,046 INFO sqlalchemy.engine.base.Engine ('xyz', 14)
2018-07-15 21:30:11,049 INFO sqlalchemy.engine.base.Engine COMMIT


In [166]:
session.dirty

IdentitySet([])

In [167]:
session.add_all([
    Album(title='1', artist_id=1),
    Album(title='2', artist_id=2),
])

In [168]:
session.dirty

IdentitySet([])

In [169]:
session.new

IdentitySet([<__main__.Album object at 0x105f1df28>, <__main__.Album object at 0x105f3a080>])

new
- 새로운 내용이 추가되었는지 감시.

In [170]:
session.commit()

2018-07-15 21:30:33,406 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-15 21:30:33,413 INFO sqlalchemy.engine.base.Engine INSERT INTO album (title, artist_id) VALUES (?, ?)
2018-07-15 21:30:33,416 INFO sqlalchemy.engine.base.Engine ('1', 1)
2018-07-15 21:30:33,422 INFO sqlalchemy.engine.base.Engine INSERT INTO album (title, artist_id) VALUES (?, ?)
2018-07-15 21:30:33,424 INFO sqlalchemy.engine.base.Engine ('2', 2)
2018-07-15 21:30:33,426 INFO sqlalchemy.engine.base.Engine COMMIT


In [171]:
session.add_all([
    Genre(name='Rock'),
    Genre(name='Metal'),
])

In [172]:
session.new

IdentitySet([<__main__.Genre object at 0x105f3a6d8>, <__main__.Genre object at 0x105f3a748>])

In [173]:
session.commit()

2018-07-15 21:30:37,900 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-15 21:30:37,903 INFO sqlalchemy.engine.base.Engine INSERT INTO genre (name) VALUES (?)
2018-07-15 21:30:37,904 INFO sqlalchemy.engine.base.Engine ('Rock',)
2018-07-15 21:30:37,907 INFO sqlalchemy.engine.base.Engine INSERT INTO genre (name) VALUES (?)
2018-07-15 21:30:37,909 INFO sqlalchemy.engine.base.Engine ('Metal',)
2018-07-15 21:30:37,911 INFO sqlalchemy.engine.base.Engine COMMIT


In [174]:
temp = Track(title='3-1', album_id=3, genre_id=3)

In [175]:
session.add_all([temp])

In [176]:
session.new

IdentitySet([<__main__.Track object at 0x105f3a198>])

In [177]:
session.commit()

2018-07-15 21:30:43,939 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-15 21:30:43,944 INFO sqlalchemy.engine.base.Engine INSERT INTO track (title, album_id, genre_id) VALUES (?, ?, ?)
2018-07-15 21:30:43,946 INFO sqlalchemy.engine.base.Engine ('3-1', 3, 3)
2018-07-15 21:30:43,950 INFO sqlalchemy.engine.base.Engine COMMIT


객체형태로 저장하여 내용을 삽입할수도 있다.

In [178]:
for row in session.query(Artist):
    print(row.id, row.name)

2018-07-15 21:31:00,182 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-15 21:31:00,185 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id, artist.name AS artist_name 
FROM artist
2018-07-15 21:31:00,187 INFO sqlalchemy.engine.base.Engine ()
1 Led Zepplin
2 xyz
3 Led Zepplin
4 xyz
5 Led Zepplin
6 xyz
7 Led Zepplin
8 xyz
9 Led Zepplin
10 xyz
11 Led Zepplin
12 xyz
13 Led Zepplin
14 xyz


query: Artist 테이블에 저장된 모든 내용 검색(=select 구문)

In [179]:
for row in session.query(Artist).filter(Artist.id==1):
    print(row.id, row.name)

2018-07-15 21:31:11,571 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id, artist.name AS artist_name 
FROM artist 
WHERE artist.id = ?
2018-07-15 21:31:11,575 INFO sqlalchemy.engine.base.Engine (1,)
1 Led Zepplin


filter: 조건 적용하여 Artist 테이블 검색. argument를 파이썬 문법에 맞게 넣어야 한다.(=where 구문)

In [181]:
for row in session.query(Artist).filter_by(name="Led Zepplin"):
    print(row.id, row.name)

2018-07-15 21:36:31,033 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id, artist.name AS artist_name 
FROM artist 
WHERE artist.name = ?
2018-07-15 21:36:31,037 INFO sqlalchemy.engine.base.Engine ('Led Zepplin',)
1 Led Zepplin
3 Led Zepplin
5 Led Zepplin
7 Led Zepplin
9 Led Zepplin
11 Led Zepplin
13 Led Zepplin


filter_by: filter와 유사한 기능. argument를 sql 문법에 맞게 넣어야 한다.(=where 구문)

In [184]:
result = session.query(Track.title, Album.title, Genre.name, Artist.name).select_from(Track).join(Album).join(Genre).join(Artist).all()

for row in result:
    print(row)
    
songList = [dict(Track=row[0], Album=row[1], Genre=row[2], Artist=row[3]) for row in result]

songList

2018-07-15 21:40:55,107 INFO sqlalchemy.engine.base.Engine SELECT track.title AS track_title, album.title AS album_title, genre.name AS genre_name, artist.name AS artist_name 
FROM track JOIN album ON album.id = track.album_id JOIN genre ON genre.id = track.genre_id JOIN artist ON artist.id = album.artist_id
2018-07-15 21:40:55,109 INFO sqlalchemy.engine.base.Engine ()
('3-1', '1', 'Rock', 'Led Zepplin')
('3-1', '1', 'Rock', 'Led Zepplin')


[{'Track': '3-1', 'Album': '1', 'Genre': 'Rock', 'Artist': 'Led Zepplin'},
 {'Track': '3-1', 'Album': '1', 'Genre': 'Rock', 'Artist': 'Led Zepplin'}]

multiple join을 실행하고 그 결과를 dict를 활용하여 list에 넣었다.

In [185]:
from sqlalchemy.orm import relationship

relationship
- 2개의 클래스 간 관계 정의.
- 부모 관계나 참조 테이블 관계가 있음.
- back_populates: 두 클래스 간 관계에서 화살표가 가는 방향은 문제가 없으나 돌아오는 방향을 모르기때문에 돌아오는 곳이 어디인지 명시해주기 위해 사용한다.

In [186]:
Base = declarative_base()

In [187]:
class Artist(Base):
    __tablename__ = "Artist"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    albumList = relationship("Album", back_populates="artist")

In [188]:
class Album(Base):
    __tablename__ = "Album"
    
    id = Column(Integer, primary_key=True)
    title = Column(String)
    artist_id = Column(Integer, ForeignKey("Artist.id"))
    
    artist = relationship("Artist", back_populates="albumList", uselist=False)
    trackList = relationship("Track", back_populates="album")

In [189]:
class Genre(Base):
    __tablename__ = "Genre"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    trackList = relationship("Track", back_populates="genre")

In [190]:
class Track(Base):
    __tablename__ = "Track"
    
    id = Column(Integer, primary_key=True)
    title = Column(String)
    length = Column(Integer)
    rating = Column(Integer)
    count = Column(Integer)
    album_id = Column(Integer, ForeignKey("Album.id"))
    genre_id = Column(Integer, ForeignKey("Genre.id"))
    
    album = relationship("Album", back_populates="trackList", uselist=False)
    genre = relationship("Genre", back_populates="trackList", uselist=False)

In [203]:
track1 = Track(id=1, title="Black Dog", rating=5, length=297, count=0)
track2 = Track(id=2, title="Stairway", rating=5, length=482, count=0)
track3 = Track(id=3, title="About to rock", rating=5, length=313, count=0)
track4 = Track(id=4, title="Who Made Who", rating=5, length=297, count=0)

track1.album = track2.album = Album(title="IV", id=1)
track3.album = track4.album = Album(title="Who Made Who", id=2)

track1.genre = track3.genre = Genre(name="Rock", id=1)
track2.genre = track4.genre = Genre(name="Metal", id=2)

track1.album.artist = track2.album.artist = Artist(name="Led Zepplin", id=1)
track3.album.artist = track4.album.artist = Artist(name="AC/DC", id=2)

In [204]:
print("Title: %s, Album: %s, Genre: %s, Artist: %s" % (track1.title, track1.album.title, track1.genre.name, track1.album.artist.name))
print("Title: %s, Album: %s, Genre: %s, Artist: %s" % (track3.title, track3.album.title, track3.genre.name, track3.album.artist.name))

Title: Black Dog, Album: IV, Genre: Rock, Artist: Led Zepplin
Title: About to rock, Album: Who Made Who, Genre: Rock, Artist: AC/DC


In [205]:
print("TrackID: %d, AlbumID: %d, GenreID: %d, ArtistID: %d" % (track1.id, track1.album.id, track1.genre.id, track1.album.artist.id))
print("TrackID: %d, AlbumID: %d, GenreID: %d, ArtistID: %d" % (track3.id, track3.album.id, track3.genre.id, track3.album.artist.id))

TrackID: 1, AlbumID: 1, GenreID: 1, ArtistID: 1
TrackID: 3, AlbumID: 2, GenreID: 1, ArtistID: 2
